In [ ]:
from ase.io import vasp
from ase import Atoms
import numpy as np 
import pandas as pd
from tabulate import tabulate

In [ ]:
def getZVals(outcar):
    atoms=None
    zvals=None
    with open(outcar,'r') as fd:
        for line in fd:
            if (atoms is None) and (" POSCAR =" in line) :
                atoms = line.rstrip().split()[2:]
            if (zvals is None) and ("   ZVAL   =" in line):
                zvals = list(map(float,line.rstrip().split()[2:]))
            if zvals and atoms:
                return dict(zip(atoms,zvals))
    raise Exception("ZVALS not found")

In [ ]:
atoms:Atoms = vasp.read_vasp("./POSCAR")
symbols = atoms.get_chemical_symbols()
baderData = pd.read_csv("./ACF.dat",skiprows=[1],delimiter=" ",skipinitialspace=True,nrows=len(atoms))

zVals=getZVals("./OUTCAR")
valanceCharge = np.array([zVals[symbol] for symbol in symbols])
baderCharge = baderData["CHARGE"].values

finalCharge = baderCharge-valanceCharge
print("Positive value : Charge Gain")
print("Negative value : Charge Loss\n")
DF = pd.DataFrame({
    'symbols':symbols,
    'ZVAL':valanceCharge,
    'Bader':baderCharge,
    'Final Charge':finalCharge
})
print(tabulate(DF,headers='keys',tablefmt='fancy_grid',floatfmt="0.2f"))